In [ ]:
import pandas as pd
import numpy as np

### Carregar tabelas de dados

In [ ]:
soja = pd.read_csv('../dados_coletados/tabela_soja.csv', na_values = '...')
pontos = pd.read_csv('../dados_coletados/municipios_coordenadas.csv')

### Juntar as coordenadas dos municípios com os dados de produção

In [ ]:
dados = pd.merge(soja, pontos, how = 'outer', 
                 left_on=['Municipio','Estado'],
                 right_on = ['Municipio','Estado'])
dados

### Verificar municípios que os dados não foram pareados corretamente

In [ ]:
soja_na = dados[dados['lat'].isna()]['Municipio'].unique()
print(soja_na)
coordenadas_na = dados[dados['Ano'].isna()]['Municipio'].unique()
print(coordenadas_na)

### Definir o método para atualizar o nome do município e coordenadas

In [ ]:
def corrigir_grafia(estado, antigo, novo):
    global dados
    dados.loc[(dados['Estado'] == estado) & (dados['Municipio'] == antigo), 'Municipio'] = novo
    coord = dados[(dados['Estado'] == estado) &
                  (dados['Municipio'] == novo) &
                  (dados['lat'].notna())][['lat', 'long']]
    dados.loc[(dados['Estado'] == estado) &
              (dados['Municipio'] == novo), 'lat'] = coord.lat.values[0]
    dados.loc[(dados['Estado'] == estado) &
              (dados['Municipio'] == novo), 'long'] = coord.long.values[0]
    dados = dados.drop(dados[((dados['Estado'] == estado) &
                              (dados['Municipio'] == novo) &
                              (dados['Ano'].isna()))].index)
    print(dados[(dados['Estado'] == estado) & (dados['Municipio'] == antigo)])
    print(dados[(dados['Estado'] == estado) & (dados['Municipio'] == novo)])

#### Augusto Severo

In [ ]:
#Augusto Severo mudou grafia para Campo Grande
#Fonte: https://www.ibge.gov.br/geociencias/organizacao-do-territorio/estrutura-territorial/27336-alteracoes-toponimicas-municipais.html?=&t=acesso-ao-produto
print(dados[(dados['Estado'] == 'RN') & (dados['Municipio'] == 'Augusto Severo')])
print(dados[(dados['Estado'] == 'RN') & (dados['Municipio'] == 'Campo Grande')])
corrigir_grafia('RN', 'Augusto Severo', 'Campo Grande')

#### Fortaleza do Tabocão

In [ ]:
#Fortaleza do Tabocão mudou grafia para Tabocão
#Fonte: https://www.ibge.gov.br/geociencias/organizacao-do-territorio/estrutura-territorial/27336-alteracoes-toponimicas-municipais.html?=&t=acesso-ao-produto
print(dados[(dados['Estado'] == 'TO') & (dados['Municipio'] == 'Fortaleza do Tabocão')])
print(dados[(dados['Estado'] == 'TO') & (dados['Municipio'] == 'Tabocão')])
corrigir_grafia('TO', 'Fortaleza do Tabocão', 'Tabocão')

#### Ererê

In [ ]:
#Ererê mudou grafia para Ereré
#Fonte: https://www.ibge.gov.br/geociencias/organizacao-do-territorio/estrutura-territorial/27336-alteracoes-toponimicas-municipais.html?=&t=acesso-ao-produto
print(dados[(dados['Estado'] == 'CE') & (dados['Municipio'] == 'Ererê')])
print(dados[(dados['Estado'] == 'CE') & (dados['Municipio'] == 'Ereré')])
corrigir_grafia('CE', 'Ererê', 'Ereré')

#### Dona Eusébia

In [ ]:
#Dona Eusébia mudou grafia para Dona Euzébia
#Fonte: https://www.ibge.gov.br/geociencias/organizacao-do-territorio/estrutura-territorial/27336-alteracoes-toponimicas-municipais.html?=&t=acesso-ao-produto
print(dados[(dados['Estado'] == 'MG') & (dados['Municipio'] == 'Dona Eusébia')])
print(dados[(dados['Estado'] == 'MG') & (dados['Municipio'] == 'Dona Euzébia')])
corrigir_grafia('MG', 'Dona Eusébia', 'Dona Euzébia')

#### Pingo d'Água

In [ ]:
#Pingo d'Água mudou grafia para Pingo-d'Água 
#Fonte: https://www.ibge.gov.br/geociencias/organizacao-do-territorio/estrutura-territorial/27336-alteracoes-toponimicas-municipais.html?=&t=acesso-ao-produto
print(dados[(dados['Estado'] == 'MG') & (dados['Municipio'] == "Pingo d'Água")])
print(dados[(dados['Estado'] == 'MG') & (dados['Municipio'] == "Pingo-d'Água")])
corrigir_grafia('MG', "Pingo d'Água", "Pingo-d'Água")

#### São Thomé das Letras

In [ ]:
#São Thomé das Letras mudou grafia para São Tomé das Letras
#Fonte: https://www.ibge.gov.br/geociencias/organizacao-do-territorio/estrutura-territorial/27336-alteracoes-toponimicas-municipais.html?=&t=acesso-ao-produto
print(dados[(dados['Estado'] == 'MG') & (dados['Municipio'] == 'São Thomé das Letras')])
print(dados[(dados['Estado'] == 'MG') & (dados['Municipio'] == 'São Tomé das Letras')])
corrigir_grafia('MG', 'São Thomé das Letras', 'São Tomé das Letras')

### Atualizar lista de municípios não pareados corretamente

In [ ]:
coordenadas_na = np.delete(coordenadas_na, 
          np.in1d(coordenadas_na,
                  ['Campo Grande', 'Tabocão', 'Ereré',
                   'Dona Euzébia', "Pingo-d'Água", 'São Tomé das Letras']))
coordenadas_na

### Verificar a inexistência de municípios sem coordenadas na tabela de dados

In [ ]:
producao_na = dados[dados['lat'].isna()]['Municipio'].unique()
print(producao_na)
#Nenhum município sem coordenadas na tabela de dados

### Remover municípios com coordenadas, mas ausentes da tabela de dados de produção

In [ ]:
len(dados[dados['Ano'].isna()])
print(dados[dados['Ano'].isna()])
#remover 9 municípios com coordenadas, mas sem dados de produção
dados = dados[dados['Ano'].notna()]
print(dados[dados['Ano'].isna()])

### Definir variável 'Ano' como tipo integrais

In [ ]:
#será útil para ordenar e extrair subconjuntos
dados['Ano'] = dados['Ano'].astype(int)
dados

### Salvar a tabela de dados

In [ ]:
dados.to_csv('../dados_coletados/dados.csv', index = False)